In [2]:
# Query World Bank Documents & Reports API for education documents in Morocco
import requests
import pandas as pd

url = "https://search.worldbank.org/api/v3/wds"
params = {
    "format": "json",
    "qterm": "education",
    "count_exact": "Morocco",
    "fl": "display_title,docdt,count",
    "rows": 20,  # Number of results per page
    "os": 0      # Start at first record
}

response = requests.get(url, params=params)
data = response.json()

# The results are in the 'documents' key
docs = data.get('documents', {})

# Convert to DataFrame
df = pd.DataFrame.from_dict(docs, orient='index')

# Display the first few results
df[['display_title', 'docdt', 'count']].head()

,display_title,docdt,count
D30441842,Morocco - Education Support Program : Concept ...,2018-09-23T04:00:00Z,Morocco
D31007494,Morocco - Education Support Program,2019-04-24T04:00:00Z,Morocco
D30996993,Morocco - Education Support Program Project : ...,2019-04-20T04:00:00Z,Morocco
D31007403,Appraisal Stage Integrated Safeguards Data She...,2019-04-24T04:00:00Z,Morocco
D19718882,Morocco - First Education Development Policy Loan,2013-12-16T05:00:00Z,Morocco


In [10]:
import wbgapi as wb
import pandas as pd

# Dictionary with valid World Bank indicators for education
indicators = {
    # Primary Education - Enrollment Rates
    'Primary GER Total': 'SE.PRM.ENRR',
    'Primary GER Male': 'SE.PRM.ENRR.MA',
    'Primary GER Female': 'SE.PRM.ENRR.FE',
    'Primary NER Total': 'SE.PRM.NENR',
    'Primary NER Male': 'SE.PRM.NENR.MA',
    'Primary NER Female': 'SE.PRM.NENR.FE',
    
    # Secondary Education - Enrollment Rates
    'Secondary GER Total': 'SE.SEC.ENRR',
    'Secondary GER Male': 'SE.SEC.ENRR.MA',
    'Secondary GER Female': 'SE.SEC.ENRR.FE',
    'Secondary NER Total': 'SE.SEC.NENR',
    'Secondary NER Male': 'SE.SEC.NENR.MA',
    'Secondary NER Female': 'SE.SEC.NENR.FE',
    
    # Tertiary Education - Enrollment Rates
    'Tertiary GER Total': 'SE.TER.ENRR',
    'Tertiary GER Male': 'SE.TER.ENRR.MA',
    'Tertiary GER Female': 'SE.TER.ENRR.FE',
    
    # Preprimary Education - Enrollment Rates
    'Preprimary GER Total': 'SE.PRE.ENRR',
    
    # Intake Rates
    'Net intake rate grade 1 total': 'SE.PRM.NINT.ZS',
    'Net intake rate grade 1 male': 'SE.PRM.NINT.MA.ZS',
    'Net intake rate grade 1 female': 'SE.PRM.NINT.FE.ZS',
    
    # Completion Rates
    'Primary Completion Rate Total': 'SE.PRM.CMPT.ZS',
    'Primary Completion Rate Male': 'SE.PRM.CMPT.MA.ZS',
    'Primary Completion Rate Female': 'SE.PRM.CMPT.FE.ZS',
    'Lower Secondary Completion Rate Total': 'SE.SEC.CMPT.LO.ZS',
    'Lower Secondary Completion Rate Male': 'SE.SEC.CMPT.LO.MA.ZS',
    'Lower Secondary Completion Rate Female': 'SE.SEC.CMPT.LO.FE.ZS',
    
    # Progression and Persistence
    'Persistence to last grade of primary total': 'SE.PRM.PRSL.ZS',
    'Persistence to last grade of primary male': 'SE.PRM.PRSL.MA.ZS',
    'Persistence to last grade of primary female': 'SE.PRM.PRSL.FE.ZS',
    'Progression to secondary school total': 'SE.SEC.PROG.ZS',
    'Progression to secondary school male': 'SE.SEC.PROG.MA.ZS',
    'Progression to secondary school female': 'SE.SEC.PROG.FE.ZS',
    
    # Repetition Rates
    'Repeaters primary all grades (%)': 'SE.PRM.REPT.ZS',
    'Repeaters primary all grades male (%)': 'SE.PRM.REPT.MA.ZS',
    'Repeaters primary all grades female (%)': 'SE.PRM.REPT.FE.ZS',
    
    # Out of School Rates
    'Out-of-school rate primary age total': 'SE.PRM.UNER.ZS',
    'Out-of-school rate primary age male': 'SE.PRM.UNER.MA.ZS',
    'Out-of-school rate primary age female': 'SE.PRM.UNER.FE.ZS',
    
    # Private Enrollment
    'Private enrollment, primary (% of total primary)': 'SE.PRM.PRIV.ZS',
    'Private enrollment, secondary (% of total secondary)': 'SE.SEC.PRIV.ZS',
    
    # Total Enrollment Numbers
    'School enrollment, primary': 'SE.PRM.ENRL',
    'School enrollment, secondary': 'SE.SEC.ENRL',
    
    # Total Teachers by Level
    'Teachers, primary education': 'SE.PRM.TCHR',
    'Teachers, primary education, female (%)': 'SE.PRM.TCHR.FE.ZS',
    'Teachers, secondary education': 'SE.SEC.TCHR',
    'Teachers, secondary education, female (%)': 'SE.SEC.TCHR.FE.ZS',
    'Teachers, tertiary education, female (%)': 'SE.TER.TCHR.FE.ZS',
    
    # Trained Teachers
    'Trained teachers in primary education (%)': 'SE.PRM.TCAQ.ZS',
    'Trained teachers in primary education, female (%)': 'SE.PRM.TCAQ.FE.ZS',
    'Trained teachers in primary education, male (%)': 'SE.PRM.TCAQ.MA.ZS',
    
    # Pupil-Teacher Ratios
    'Pupil-teacher ratio, primary': 'SE.PRM.ENRL.TC.ZS',
    'Pupil-teacher ratio, secondary': 'SE.SEC.ENRL.TC.ZS',
    'Pupil-teacher ratio, tertiary': 'SE.TER.ENRL.TC.ZS',
    'Pupil-teacher ratio, preprimary': 'SE.PRE.ENRL.TC.ZS',
    
    # Education Staff Compensation
    'All education staff compensation, primary (%)': 'SE.XPD.PRIM.PC.ZS',
    'All education staff compensation, secondary (%)': 'SE.XPD.SECO.PC.ZS',
    # Total Education Expenditure
    'Government expenditure on education, total (% of GDP)': 'SE.XPD.TOTL.GD.ZS',
    'Government expenditure on education, total (% of government expenditure)': 'SE.XPD.TOTL.GB.ZS',
    
    # Current vs Capital Expenditure
    'Current education expenditure, primary (% of total expenditure in primary public institutions)': 'SE.XPD.CPRM.ZS',
    'Current education expenditure, secondary (% of total expenditure in secondary public institutions)': 'SE.XPD.CSEC.ZS',
    
    
    
    'GDP per capita (current US$)': 'NY.GDP.PCAP.CD',
    'GDP per capita, PPP (current international $)': 'NY.GDP.PCAP.PP.CD',
    'GDP per capita, PPP (constant 2017 international $)': 'NY.GDP.PCAP.PP.KD',
    
    'Household final consumption expenditure (% of GDP)': 'NE.CON.PRVT.ZS',  # General household consumption, not education-specific
    
    
    # TOTAL EXPENDITURE (may include both public and private in some countries)
    'Adjusted savings: education expenditure (% of GNI)': 'NY.ADJ.AEDU.GN.ZS',
    
    # LITERACY RATES (as a quality outcome)
    'Literacy rate, youth male (% of males ages 15-24)': 'SE.ADT.1524.LT.MA.ZS',
    
    
    # YOUTH UNEMPLOYMENT RATES
    'Unemployment, youth total (% of total labor force ages 15-24)': 'SL.UEM.1524.ZS',
    'Unemployment, youth male (% of male labor force ages 15-24)': 'SL.UEM.1524.MA.ZS',
    'Unemployment, youth female (% of female labor force ages 15-24)': 'SL.UEM.1524.FE.ZS',
    'Labor force, total': 'SL.TLF.TOTL.IN',
    
    # UNEMPLOYMENT BY EDUCATION LEVEL
    'Unemployment with basic education (% of total labor force with basic education)': 'SL.UEM.BASC.ZS',
    'Unemployment with intermediate education (% of total labor force with intermediate education)': 'SL.UEM.INTM.ZS',
    'Unemployment with basic education, male (% of male labor force with basic education)': 'SL.UEM.BASC.MA.ZS',
    'Unemployment with basic education, female (% of female labor force with basic education)': 'SL.UEM.BASC.FE.ZS',
    'Unemployment with intermediate education, male (% of male labor force with intermediate education)': 'SL.UEM.INTM.MA.ZS',
    'Unemployment with intermediate education, female (% of female labor force with intermediate education)': 'SL.UEM.INTM.FE.ZS',
    'Unemployment with advanced education, male (% of male labor force with advanced education)': 'SL.UEM.ADVN.MA.ZS',
    'Unemployment with advanced education, female (% of female labor force with advanced education)': 'SL.UEM.ADVN.FE.ZS',
    
    # LABOR FORCE PARTICIPATION BY EDUCATION
    'Labor force with basic education (% of total working-age population with basic education)': 'SL.TLF.BASC.ZS',
    'Labor force with intermediate education (% of total working-age population with intermediate education)': 'SL.TLF.INTM.ZS',
    
    # EMPLOYMENT BY SECTOR
    'Employment in agriculture (% of total employment)': 'SL.AGR.EMPL.ZS',
    'Employment in industry (% of total employment)': 'SL.IND.EMPL.ZS',
    'Employment in services (% of total employment)': 'SL.SRV.EMPL.ZS',
    'Employment in agriculture, male (% of male employment)': 'SL.AGR.EMPL.MA.ZS',
    'Employment in agriculture, female (% of female employment)': 'SL.AGR.EMPL.FE.ZS',
    'Employment in industry, male (% of male employment)': 'SL.IND.EMPL.MA.ZS',
    'Employment in industry, female (% of female employment)': 'SL.IND.EMPL.FE.ZS',
    'Employment in services, male (% of male employment)': 'SL.SRV.EMPL.MA.ZS',
    'Employment in services, female (% of female employment)': 'SL.SRV.EMPL.FE.ZS',
    
    # SKILLS AND OCCUPATION TYPES
    'Researchers in R&D (per million people)': 'SP.POP.SCIE.RD.P6',
    'Technicians in R&D (per million people)': 'SP.POP.TECH.RD.P6',
    
    # VULNERABLE EMPLOYMENT
    'Vulnerable employment, total (% of total employment)': 'SL.EMP.VULN.ZS',
    'Vulnerable employment, male (% of male employment)': 'SL.EMP.VULN.MA.ZS',
    'Vulnerable employment, female (% of female employment)': 'SL.EMP.VULN.FE.ZS',
    
    # WAGE AND EARNINGS (limited in World Bank)
    'Wage and salaried workers, total (% of total employment)': 'SL.EMP.WORK.ZS',
    'Wage and salaried workers, male (% of male employment)': 'SL.EMP.WORK.MA.ZS',
    'Wage and salaried workers, female (% of female employment)': 'SL.EMP.WORK.FE.ZS',
    
    # GENERAL UNEMPLOYMENT
    'Unemployment, total (% of total labor force)': 'SL.UEM.TOTL.ZS',
    'Unemployment, male (% of male labor force)': 'SL.UEM.TOTL.MA.ZS',
    'Unemployment, female (% of female labor force)': 'SL.UEM.TOTL.FE.ZS',
    
    
    # ICT and Digital Skills
    'Individuals using the Internet (% of population)': 'IT.NET.USER.ZS',
    'Fixed broadband subscriptions (per 100 people)': 'IT.NET.BBND.P2',
    'High-technology exports (% of manufactured exports)': 'TX.VAL.TECH.MF.ZS',
    
    # Innovation and Knowledge Economy
    'Patent applications, residents': 'IP.PAT.RESD',
    'Scientific and technical journal articles': 'IP.JRN.ARTC.SC',
    'Gross expenditure on R&D (% of GDP)': 'GB.XPD.RSDV.GD.ZS',
    
    # Economic Complexity (relevant for skills demand)
    'GDP per person employed (constant 2017 PPP $)': 'SL.GDP.PCAP.EM.KD',
    
    
    # POPULATION BY AGE COHORTS (absolute numbers)
    'Population ages 0-14 (% of total population)': 'SP.POP.0014.TO.ZS',
    'Population, total': 'SP.POP.TOTL',
    'Population ages 10-14, Male (% of Male population)': 'SP.POP.1014.MA.5Y',
    'Population ages 10-14, female (% of female population)': 'SP.POP.1014.FE.5Y',
    'Population ages 15-19, male (% of male population)': 'SP.POP.1519.MA.5Y',
    'Population ages 15-19, Female (% of Female population)': 'SP.POP.1519.FE.5Y',
    'Population ages 20-24, male (% of male population)': 'SP.POP.2024.MA.5Y',
    'Population ages 20-24, Female (% of Female population)': 'SP.POP.2024.FE.5Y',
    'Population ages 25-29, male (% of male population)': 'SP.POP.2529.MA.5Y',
    'Population ages 25-29, female (% of female population)': 'SP.POP.2529.FE.5Y',
    
    # POPULATION BY AGE COHORTS (as % of total)
    'Population ages 15-64 (% of total population)': 'SP.POP.1564.TO.ZS',
    'Population ages 65 and above (% of total population)': 'SP.POP.65UP.TO.ZS',
    
    # TOTAL POPULATION AND GROWTH
    'Population, total': 'SP.POP.TOTL',
    'Population growth (annual %)': 'SP.POP.GROW',
    'Population density (people per sq. km of land area)': 'EN.POP.DNST',
    
    # FERTILITY INDICATORS
    'Fertility rate, total (births per woman)': 'SP.DYN.TFRT.IN',
    'Birth rate, crude (per 1,000 people)': 'SP.DYN.CBRT.IN',
    'Adolescent fertility rate (births per 1,000 women ages 15-19)': 'SP.ADO.TFRT',
    'Births attended by skilled health staff (% of total)': 'SH.STA.BRTC.ZS',
    
    # URBANIZATION INDICATORS
    'Urban population (% of total population)': 'SP.URB.TOTL.IN.ZS',
    'Urban population': 'SP.URB.TOTL',
    'Urban population growth (annual %)': 'SP.URB.GROW',
    'Rural population (% of total population)': 'SP.RUR.TOTL.ZS',
    'Rural population': 'SP.RUR.TOTL',
    'Rural population growth (annual %)': 'SP.RUR.TOTL.ZG',
    'Population in largest city': 'EN.URB.LCTY',
    'Population in urban agglomerations of more than 1 million (% of total population)': 'EN.URB.MCTY.TL.ZS',
    
    # MIGRATION INDICATORS
    'Net migration': 'SM.POP.NETM',
    'Personal remittances, received (% of GDP)': 'BX.TRF.PWKR.DT.GD.ZS',
    'Personal remittances, received (current US$)': 'BX.TRF.PWKR.CD',
    
    # GDP AND ECONOMIC GROWTH
    'GDP (current US$)': 'NY.GDP.MKTP.CD',
    'GDP (constant 2015 US$)': 'NY.GDP.MKTP.KD',
    'GDP growth (annual %)': 'NY.GDP.MKTP.KD.ZG',
    'GDP per capita (constant 2015 US$)': 'NY.GDP.PCAP.KD',
    'GDP per capita growth (annual %)': 'NY.GDP.PCAP.KD.ZG',
    'GNI per capita, Atlas method (current US$)': 'NY.GNP.PCAP.CD',
    
    # INFLATION (CPI)
    'Inflation, consumer prices (annual %)': 'FP.CPI.TOTL.ZG',
    'Consumer price index (2010 = 100)': 'FP.CPI.TOTL',
    'Inflation, GDP deflator (annual %)': 'NY.GDP.DEFL.KD.ZG',
    
    # ADDITIONAL ECONOMIC INDICATORS
    'Final consumption expenditure (% of GDP)': 'NE.CON.TOTL.ZS',
    'Government final consumption expenditure (% of GDP)': 'NE.CON.GOVT.ZS',
    'Gross capital formation (% of GDP)': 'NE.GDI.TOTL.ZS',
    'Foreign direct investment, net inflows (% of GDP)': 'BX.KLT.DINV.WD.GD.ZS',
    
    # DEPENDENCY RATIOS (important for education planning)
    'Age dependency ratio (% of working-age population)': 'SP.POP.DPND',
    'Age dependency ratio, young (% of working-age population)': 'SP.POP.DPND.YG',
    'Age dependency ratio, old (% of working-age population)': 'SP.POP.DPND.OL',
    
    # REMOTE LEARNING COVERAGE AND ACCESS
    'Access to electricity (% of population)': 'EG.ELC.ACCS.ZS',
    'Mobile cellular subscriptions (per 100 people)': 'IT.CEL.SETS.P2',
    
    # LEARNING LOSS INDICATORS
    'Learning poverty (%)': 'SE.LPV.PRIM',
    'Learning poverty, male (%)': 'SE.LPV.PRIM.MA',
    'Learning poverty, female (%)': 'SE.LPV.PRIM.FE',
    
    # COVID HEALTH IMPACT (affects education)
    'Hospital beds (per 1,000 people)': 'SH.MED.BEDS.ZS',
    
    # Government response capacity
    'Government effectiveness: Estimate': 'GE.EST',
    'Domestic general government health expenditure (% of GDP)': 'SH.XPD.GHED.GD.ZS',
    
    
    # Very few education indicators have urban/rural splits in World Bank
    'Access to electricity, rural (% of rural population)': 'EG.ELC.ACCS.RU.ZS',
    'Access to electricity, urban (% of urban population)': 'EG.ELC.ACCS.UR.ZS',
    
    
    }

country_code = 'MAR'
years = range(2000, 2025)

# Collect data with better error handling
data = {}
successful = []
failed = []

print(f"Fetching education data for Morocco ({country_code}) from {min(years)} to {max(years)}...\n")

for label, indicator_code in indicators.items():
    try:
        # Try to fetch the data
        df = wb.data.DataFrame(
            indicator_code, 
            economy=country_code, 
            time=years, 
            labels=True,
            skipBlanks=True  # Skip missing values
        )
        
        if df is not None and not df.empty:
            df = df.T  # Transpose 
            df.columns = [label]
            data[label] = df
            successful.append(label)
            print(f"✓ {label}")
        else:
            failed.append((label, "No data returned"))
            print(f"✗ {label} - No data available")
            
    except Exception as e:
        error_msg = str(e)
        if "JSON decoding error" in error_msg:
            failed.append((label, "Invalid indicator code"))
        else:
            failed.append((label, error_msg[:50]))
        print(f"✗ {label} - {indicator_code}")

# Process and display results
if data:
    # Merge all dataframes
    enrolment_data = pd.concat(data.values(), axis=1)
    enrolment_data.index.name = 'Year'
    
    # Convert to numeric
    enrolment_data = enrolment_data.apply(pd.to_numeric, errors='coerce')
    
    # Drop completely empty rows
    enrolment_data = enrolment_data.dropna(how='all')
    
    # Remove any duplicate columns
    enrolment_data = enrolment_data.loc[:, ~enrolment_data.columns.duplicated()]
    
    # Display summary statistics
    print("\n" + "="*70)
    print(f"SUMMARY: Successfully retrieved {len(successful)}/{len(indicators)} indicators")
    print(f"Years with data: {enrolment_data.index.min()} - {enrolment_data.index.max()}")
    print(f"Data shape: {enrolment_data.shape[0]} years × {enrolment_data.shape[1]} indicators")
    print("="*70)
    
    # Show data coverage
    print("\nData availability by indicator:")
    coverage = enrolment_data.notna().sum().sort_values(ascending=False)
    for ind, count in coverage.items():
        print(f"  {ind}: {count} years ({count/len(enrolment_data)*100:.1f}%)")
    
    print("\n" + "="*70)
    print("DATA PREVIEW:")
    print("="*70)
    print(enrolment_data.head(10))
    
    # Optional: Save to CSV
    enrolment_data.to_csv('morocco_education_indicators.csv')
    
    # Return the dataframe for further use
    result = enrolment_data
    
else:
    print("\nNo data could be retrieved. Check your internet connection or country code.")
    result = pd.DataFrame()

# Show failed indicators if any
if failed:
    print("\n" + "="*70)
    print(f"FAILED INDICATORS ({len(failed)}):")
    for label, reason in failed:
        print(f"  - {label}: {reason}")

# Display the final dataframe
if not result.empty:
    print("\n" + "="*70)
    print("FULL DATASET:")
    print("="*70)
    display(result)

Fetching education data for Morocco (MAR) from 2000 to 2024...

✓ Primary GER Total
✓ Primary GER Male
✓ Primary GER Female
✓ Primary NER Total
✓ Primary NER Male
✓ Primary NER Female
✓ Secondary GER Total
✓ Secondary GER Male
✓ Secondary GER Female
✓ Secondary NER Total
✓ Secondary NER Male
✓ Secondary NER Female
✓ Tertiary GER Total
✓ Tertiary GER Male
✓ Tertiary GER Female
✓ Preprimary GER Total
✓ Net intake rate grade 1 total
✓ Net intake rate grade 1 male
✓ Net intake rate grade 1 female
✓ Primary Completion Rate Total
✓ Primary Completion Rate Male
✓ Primary Completion Rate Female
✓ Lower Secondary Completion Rate Total
✓ Lower Secondary Completion Rate Male
✓ Lower Secondary Completion Rate Female
✓ Persistence to last grade of primary total
✓ Persistence to last grade of primary male
✓ Persistence to last grade of primary female
✓ Progression to secondary school total
✓ Progression to secondary school male
✓ Progression to secondary school female
✓ Repeaters primary all grades 

,Primary GER Total,Primary GER Male,Primary GER Female,Primary NER Total,Primary NER Male,Primary NER Female,Secondary GER Total,Secondary GER Male,Secondary GER Female,Secondary NER Total,...,Access to electricity (% of population),Mobile cellular subscriptions (per 100 people),Learning poverty (%),"Learning poverty, male (%)","Learning poverty, female (%)","Hospital beds (per 1,000 people)",Government effectiveness: Estimate,Domestic general government health expenditure (% of GDP),"Access to electricity, rural (% of rural population)","Access to electricity, urban (% of urban population)"
Year,,,,,,,,,,,,,,,,,,,,,
YR2000,92.925392,100.256310,85.246819,76.57579,80.83956,72.11288,39.111370,43.534260,34.579819,NaN,...,69.8,8.23968,NaN,NaN,NaN,0.88,0.026066,0.936467,44.4,92.1
YR2001,97.701721,103.781090,91.326897,80.44593,83.69017,77.04938,40.878300,45.379848,36.267170,NaN,...,71.6,16.56010,66.913338,69.483063,63.826794,0.88,NaN,1.002735,47.3,92.5
YR2002,103.055443,108.317093,97.527649,84.58196,87.23893,81.80179,42.856819,47.362808,38.234749,NaN,...,73.3,21.22970,NaN,NaN,NaN,0.88,-0.195086,1.081231,50.3,92.9
YR2003,105.895279,110.599876,100.943192,85.84979,88.07186,83.52307,44.626381,48.803822,40.324478,NaN,...,75.1,24.88850,NaN,NaN,NaN,0.89,-0.215569,0.997906,53.4,93.4
YR2004,106.186813,110.678688,101.445267,85.71852,87.73064,83.60875,47.590271,51.609058,43.428028,NaN,...,78.2,31.17170,NaN,NaN,NaN,0.87,-0.224267,1.010858,58.4,94.6
YR2005,106.112343,110.794861,101.164581,86.73812,88.72739,84.64945,49.374180,52.956322,45.637009,40.14310,...,78.6,40.82200,NaN,NaN,NaN,0.86,-0.403398,1.102531,59.3,94.3
YR2006,105.147179,109.826263,100.194550,87.99563,89.91578,85.97781,52.463760,55.873669,48.896980,42.48918,...,96.5,52.01210,74.474487,76.783913,71.902832,0.86,-0.269641,1.301862,93.0,99.3
YR2007,105.761902,110.013863,101.253036,89.26677,90.91039,87.54116,55.595711,59.019730,52.012348,45.26555,...,82.2,64.22430,NaN,NaN,NaN,0.87,-0.342614,1.416203,65.4,95.3
YR2008,104.580727,108.259079,100.674911,89.83254,91.21823,88.37824,57.887829,61.610569,53.994202,47.38257,...,84.1,72.18610,NaN,NaN,NaN,0.86,-0.356556,1.501520,68.5,95.9
